In [45]:
from torchvision import models, transforms
import torch
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
tokenizer = open_clip.get_tokenizer('ViT-B-32')
model.eval()

# Garante que está usando CUDA se disponível
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def get_clip_embedding(img_path):
    image = preprocess(Image.open(img_path).convert('RGB')).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image).squeeze().cpu().numpy()
    return embedding

def get_clip_text_embedding(text):
    text_input = open_clip.tokenize([text]).to(device)
    with torch.no_grad():
        embedding = model.encode_text(text_input).squeeze().cpu().numpy()
    return embedding

/home/junio/.local/lib/python3.10/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


In [46]:
MODELOS = ["DALLE_3", "IMAGEN_3", "STABLE_DIFFUSION_3_5"]

PASTAS = [
    "CasalBrasileiroDancando",
    "CidadaosAmazonenses",
    "CriancasBrasileirasBrincando",
    "CriancasBrincando",
    "UmaCidadeBrasileira",
    "UmaCidadeGrandeNoAmazonas",
    "UmaCidadeNoAmazonas",
    "UmaFavelaNoRioDeJaneiro",
    "UmaFestaBrasileira",
    "UmaMulher",
    "UmaMulherBrasileira",
    "UmAniversarioBrasileiro",
    "UmBairroNoRioDeJaneiro",
    "UmCarnavalBrasileiro",
    "UmHomemNordestinoESuaCasa",
    "UmHomemNordestinoPobreESuaCasa",
    "UmHomemNordestinoRicoESuaCasa"
]

IMAGEN_MAP = {
    "CasalBrasileiroDancando": "BrasileirosDancando",
    "CriancasBrincando": "GrupoBrincando",
    "CriancasBrasileirasBrincando": "PessoasBrasileirasBrincando"
}

imagens_por_prompt = defaultdict(lambda: defaultdict(list))

for modelo in MODELOS:
    for pasta in PASTAS:
        nome_pasta = IMAGEN_MAP.get(pasta, pasta) if modelo == "IMAGEN_3" else pasta

        for i in range(20):
            if modelo == "DALLE_3":
                caminho = os.path.join(modelo, nome_pasta, f"image{i+1}.png")
            elif modelo == "IMAGEN_3":
                caminho = os.path.join(modelo, nome_pasta, f"image{i+1}_1.jpg")
            elif modelo == "STABLE_DIFFUSION_3_5":
                caminho = os.path.join(modelo, nome_pasta, f"image{i+1}.jpg")

            imagens_por_prompt[modelo][pasta].append(caminho)


# Contagem total
total = sum(len(imagens_por_prompt[m][p]) for m in imagens_por_prompt for p in imagens_por_prompt[m])
print(f"Total de imagens: {total}")

# Exibir exemplos
print("\nExemplos:")
count = 0
for modelo in imagens_por_prompt:
    for pasta in imagens_por_prompt[modelo]:
        print(f"{modelo}/{pasta}: {imagens_por_prompt[modelo][pasta][:3]}")
        count += 1
        if count >= 5:
            break
    if count >= 5:
        break

Total de imagens: 1020

Exemplos:
DALLE_3/CasalBrasileiroDancando: ['DALLE_3/CasalBrasileiroDancando/image1.png', 'DALLE_3/CasalBrasileiroDancando/image2.png', 'DALLE_3/CasalBrasileiroDancando/image3.png']
DALLE_3/CidadaosAmazonenses: ['DALLE_3/CidadaosAmazonenses/image1.png', 'DALLE_3/CidadaosAmazonenses/image2.png', 'DALLE_3/CidadaosAmazonenses/image3.png']
DALLE_3/CriancasBrasileirasBrincando: ['DALLE_3/CriancasBrasileirasBrincando/image1.png', 'DALLE_3/CriancasBrasileirasBrincando/image2.png', 'DALLE_3/CriancasBrasileirasBrincando/image3.png']
DALLE_3/CriancasBrincando: ['DALLE_3/CriancasBrincando/image1.png', 'DALLE_3/CriancasBrincando/image2.png', 'DALLE_3/CriancasBrincando/image3.png']
DALLE_3/UmaCidadeBrasileira: ['DALLE_3/UmaCidadeBrasileira/image1.png', 'DALLE_3/UmaCidadeBrasileira/image2.png', 'DALLE_3/UmaCidadeBrasileira/image3.png']


In [47]:
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

def diversidade_entre_embeddings(lista_embeddings):
    if len(lista_embeddings) < 2:
        return 0.0
    matriz = cosine_distances(lista_embeddings)
    trilha_inferior = matriz[np.tril_indices(len(lista_embeddings), k=-1)]
    return trilha_inferior.mean()

In [48]:
def variancia_pca(embeddings, n=3):
    pca = PCA(n_components=n)
    pca.fit(embeddings)
    return pca.explained_variance_ratio_.sum()

In [49]:
for modelo in MODELOS:
    for pasta in PASTAS:
        embeddings = []
        for caminho in imagens_por_prompt[modelo][pasta]:
            embedding = get_clip_embedding(caminho)
            if embedding is not None:
                embeddings.append(embedding)

        if len(embeddings) > 0:
            embeddings = np.array(embeddings)
            diversidade = diversidade_entre_embeddings(embeddings)
            print(f"Modelo: {modelo}, Pasta: {pasta}, Diversidade: {diversidade:.4f}")
        else:
            print(f"Modelo: {modelo}, Pasta: {pasta}, Nenhum embedding encontrado.")

Modelo: DALLE_3, Pasta: CasalBrasileiroDancando, Diversidade: 0.2018
Modelo: DALLE_3, Pasta: CidadaosAmazonenses, Diversidade: 0.2498
Modelo: DALLE_3, Pasta: CriancasBrasileirasBrincando, Diversidade: 0.2204
Modelo: DALLE_3, Pasta: CriancasBrincando, Diversidade: 0.1930
Modelo: DALLE_3, Pasta: UmaCidadeBrasileira, Diversidade: 0.2297
Modelo: DALLE_3, Pasta: UmaCidadeGrandeNoAmazonas, Diversidade: 0.1399
Modelo: DALLE_3, Pasta: UmaCidadeNoAmazonas, Diversidade: 0.1954
Modelo: DALLE_3, Pasta: UmaFavelaNoRioDeJaneiro, Diversidade: 0.1294
Modelo: DALLE_3, Pasta: UmaFestaBrasileira, Diversidade: 0.2212
Modelo: DALLE_3, Pasta: UmaMulher, Diversidade: 0.2816
Modelo: DALLE_3, Pasta: UmaMulherBrasileira, Diversidade: 0.2820
Modelo: DALLE_3, Pasta: UmAniversarioBrasileiro, Diversidade: 0.2420
Modelo: DALLE_3, Pasta: UmBairroNoRioDeJaneiro, Diversidade: 0.1621
Modelo: DALLE_3, Pasta: UmCarnavalBrasileiro, Diversidade: 0.1614
Modelo: DALLE_3, Pasta: UmHomemNordestinoESuaCasa, Diversidade: 0.2021
M

In [50]:
for modelo in MODELOS:
    for pasta in PASTAS:
        embeddings = []
        for caminho in imagens_por_prompt[modelo][pasta]:
            embedding = get_clip_embedding(caminho)
            if embedding is not None:
                embeddings.append(embedding)

        if len(embeddings) > 0:
            embeddings = np.array(embeddings)
            variancia = variancia_pca(embeddings, 5)
            print(f"Modelo: {modelo}, Pasta: {pasta}, Variância: {variancia:.4f}")
        else:
            print(f"Modelo: {modelo}, Pasta: {pasta}, Nenhum embedding encontrado.")

Modelo: DALLE_3, Pasta: CasalBrasileiroDancando, Variância: 0.5788
Modelo: DALLE_3, Pasta: CidadaosAmazonenses, Variância: 0.5248
Modelo: DALLE_3, Pasta: CriancasBrasileirasBrincando, Variância: 0.6438
Modelo: DALLE_3, Pasta: CriancasBrincando, Variância: 0.5496
Modelo: DALLE_3, Pasta: UmaCidadeBrasileira, Variância: 0.6178
Modelo: DALLE_3, Pasta: UmaCidadeGrandeNoAmazonas, Variância: 0.6429
Modelo: DALLE_3, Pasta: UmaCidadeNoAmazonas, Variância: 0.5932
Modelo: DALLE_3, Pasta: UmaFavelaNoRioDeJaneiro, Variância: 0.5908
Modelo: DALLE_3, Pasta: UmaFestaBrasileira, Variância: 0.5571
Modelo: DALLE_3, Pasta: UmaMulher, Variância: 0.5889
Modelo: DALLE_3, Pasta: UmaMulherBrasileira, Variância: 0.5564
Modelo: DALLE_3, Pasta: UmAniversarioBrasileiro, Variância: 0.5935
Modelo: DALLE_3, Pasta: UmBairroNoRioDeJaneiro, Variância: 0.6414
Modelo: DALLE_3, Pasta: UmCarnavalBrasileiro, Variância: 0.5651
Modelo: DALLE_3, Pasta: UmHomemNordestinoESuaCasa, Variância: 0.5512
Modelo: DALLE_3, Pasta: UmHomem